In [191]:
# home team         away team                                    season,datetime,referee
# fthg-predict      ftag-predict        **ftr-sanity check
# hthg              htag                **half time result
# hs                as
# hst               ast
# hc                ac
# hf                af
# hy                ay
# hr                ar  
# ftr_num
# htr-num
 

## Feature eng

In [192]:
import pandas as pd 
data_path=r"epl_23-f.csv"
data=pd.read_csv(data_path,encoding='windows-1254')
data.columns=[i.lower() for i in data.columns]
data.datetime=pd.to_datetime(data.datetime).dt.date  # dates were in string format do converted to datetime format/some variant
data.drop(data.loc[data['season'].str.contains('1993|1994|1995|1996|1997|1998|1999',regex=True)].index,inplace=True) #contains null values
# always make them in this order  to keep the ordering a constant
data.sort_values(['datetime','hometeam','awayteam','referee'],ascending=True,inplace=True) #this be the unique order
data.reset_index(drop=True,inplace=True) #and resetting index to make it starting from zero again
# no matter what happens i always want the above code to be running
# ammona keep this in mind ammona make this all features as i am predicting just one thing which is full time results-ftr




def transform1(data):
    """some tranasformations and features"""
    points_h_map={'H':3,'D':1,'A':0}
    points_a_map={'H':0,'D':1,'A':3}
    data['hp']=data['ftr'].map(points_h_map)
    data['ap']=data['ftr'].map(points_a_map)
    # win_map={'H':0,'D':1,'A':2}
    # data['ftr_num']=data['ftr'].map(win_map) #point home team got in that match
    # data['htr_num']=data['htr'].map(win_map) #points awa team got in that match
    return data
data=transform1(data)

In [193]:
columns_to_sort_by=data.select_dtypes('object').drop(['season','ftr','htr'],axis=1).columns.tolist()
columns_to_taransform=data.select_dtypes(include=['float64','int64']).columns.tolist()


In [194]:
# (data.groupby('datetime',as_index=False)['referee'].value_counts()['count']!=1).sum()
object_d=list(data.select_dtypes('object').drop(['hometeam','awayteam'],axis=1).columns) #[:7]
# int_d=['ftr_num','htr_num'] #[7:9]
home=['hometeam','fthg','hthg','hs','hst','hc','hf','hy','hr','hp'] #[9:18]-2
away=['awayteam','ftag','htag','as','ast','ac','af','ay','ar','ap'] #[18:]-2
data=data.reindex(columns=object_d+home+away)   #**very important data -the base data

import numpy as np
home_data=data[object_d+home]# data with only home attributes
away_data=data[object_d+away] #data with only away attributes
original_features_names=['ftg','htg','s','st','c','f','y','r','p']

In [196]:
# rolling home and away combined
def rol_hac(home_data,away_data):
    """making a rolling of all attributes representing a team offense"""

    away_data.columns=object_d+home #changed away to look like home
    combined_data=pd.concat([home_data,away_data],ignore_index=True)#cancatted it
    combined_data['identification']=np.array([np.ones(home_data.shape[0]),np.zeros(home_data.shape[0])],dtype=np.int64).reshape(-1,1)#making an identifier to split later
    # combined data-very important its data raveled vertically
    
    rol_hac=pd.DataFrame(index=combined_data.index)
    features_to_rol=home[1:]
    for col in features_to_rol:
        feature=combined_data.groupby('hometeam',as_index=False)[col].rolling(window=10,center=True,min_periods=5).mean().shift(1).fillna(method='bfill')[col]
        rol_hac[col]=feature
    home_df=rol_hac[combined_data.identification==1]
    away_df=rol_hac[combined_data.identification==0].reset_index(drop=True)
    home_df.columns=[i+'_hr'for i in original_features_names]
    away_df.columns=[i+'_ar'for i in original_features_names]
    rol_features=home_df.join(away_df) #suffixes are for overlapping columns
    return rol_features

def rol_hac_d(home_data,away_data):
    """this features are rolling home and away combined attributes(attributes like home shots,home fouls away yellow acrds etc )
    conceded by home and away teams until the previous match
    hs_c=shots conceded by the  home team until the previous match
    ha_c=shots conceded by the away team until the previous match ,i think you get the idea
    """
    
    home_data.columns=object_d+away #changed away to look like home
    combined_data=pd.concat([away_data,home_data],ignore_index=True)#cancatted it
    combined_data['identification']=np.array([np.ones(home_data.shape[0]),np.zeros(home_data.shape[0])],dtype=np.int64).reshape(-1,1)#making an identifier to split later
    # ones represent what is home teams defence and zeros represent away teams defence
   
    rol_hac=pd.DataFrame(index=combined_data.index)
    features_to_rol=away[1:]
    for col in features_to_rol:
        feature=combined_data.groupby('awayteam',as_index=False)[col].rolling(window=10,center=True,min_periods=5).mean().shift(1).fillna(method='bfill')[col]
        rol_hac[col]=feature # c indicates conceded
    home_df=rol_hac[combined_data.identification==1]
    home_df.columns=[i+'_chr'for i in original_features_names] #-chr-conceded home rolling
    away_df=rol_hac[combined_data.identification==0].reset_index(drop=True)
    away_df.columns=[i+'_car'for i in original_features_names] #-car-conceded away rolling
    rol_features=home_df.join(away_df)
    return rol_features

In [197]:
rollin_features=rol_hac(home_data.copy(),away_data.copy())
# rolling features are rolling attributes of team when they were home and away combined

rollin_features_d=rol_hac_d(home_data.copy(),away_data.copy())
# rolling_features_d are rolling attributes conceded by teams when they were home and away, combined

In [209]:
ob_data=data.select_dtypes('object')
new_data=ob_data.join([rollin_features,rollin_features_d])
new_data

,season,datetime,ftr,htr,referee,hometeam,awayteam,ftg_hr,htg_hr,s_hr,...,p_chr,ftg_car,htg_car,s_car,st_car,c_car,f_car,y_car,r_car,p_car
0,2000-01,2000-08-19,H,H,Rob Harris,Charlton,Man City,0.666667,0.333333,11.000000,...,3.000000,1.400000,0.400000,9.300000,5.200000,4.800000,10.400000,1.200000,0.200000,1.100000
1,2000-01,2000-08-19,H,H,Graham Barber,Chelsea,West Ham,1.000000,0.166667,7.833333,...,0.833333,2.400000,1.100000,13.400000,6.300000,5.700000,12.800000,1.700000,0.000000,2.000000
2,2000-01,2000-08-19,A,D,Barry Knight,Coventry,Middlesbrough,2.500000,1.166667,13.000000,...,2.000000,0.900000,0.700000,9.700000,3.200000,5.300000,16.200000,2.900000,0.100000,0.500000
3,2000-01,2000-08-19,D,A,Andy D'Urso,Derby,Southampton,1.500000,0.666667,8.666667,...,1.500000,1.100000,0.400000,7.900000,4.500000,5.100000,14.100000,1.600000,0.100000,0.400000
4,2000-01,2000-08-19,H,H,Dermot Gallagher,Leeds,Everton,0.833333,0.000000,8.000000,...,0.333333,1.700000,0.600000,11.800000,4.700000,6.400000,12.900000,1.700000,0.000000,1.300000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8284,2021-22,2022-04-09,A,A,A Marriner,Watford,Leeds,0.600000,0.300000,11.000000,...,1.300000,0.571429,0.428571,10.428571,3.285714,3.714286,10.714286,1.142857,0.142857,0.000000
8285,2021-22,2022-04-10,H,D,M Atkinson,Brentford,West Ham,1.100000,0.400000,11.400000,...,0.900000,1.142857,0.142857,11.714286,3.857143,4.285714,8.142857,1.285714,0.142857,1.428571
8286,2021-22,2022-04-10,H,H,R Jones,Leicester,Crystal Palace,1.400000,0.700000,10.300000,...,0.900000,1.571429,0.857143,11.285714,3.285714,3.285714,8.571429,0.714286,0.000000,2.000000
8287,2021-22,2022-04-10,D,H,A Taylor,Man City,Liverpool,2.800000,1.500000,15.500000,...,2.300000,3.428571,1.857143,18.571429,8.000000,8.714286,8.285714,0.428571,0.000000,2.285714


In [235]:
from sklearn.feature_selection import mutual_info_classif
pp=new_data2.select_dtypes('number')
tx=new_data2['ftr']
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
tx=le.fit_transform(tx)
mi=mutual_info_classif(pp,tr)

import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
plt.figure(figsize=(5,15))
mi=pd.DataFrame(mi,index=pp.columns,columns=['mi score'])
# sns.barplot(mi['mi score'],mi.index)

mi.sort_values('mi score',ascending=False)

,mi score
p_ar,0.116341
p_car,0.114309
p_hr,0.105313
p_chr,0.102079
ftg_car,0.073105
ftg_chr,0.062512
ftg_hr,0.057973
ftg_ar,0.057822
htg_hr,0.045840
s_chr,0.037081


<Figure size 360x1080 with 0 Axes>

In [233]:
import warnings
warnings.simplefilter('ignore')
import preprocessing
import features
import optimization
import utility
import models

train,test=utility.splits(new_data2,test_size=0.2)
train,val=utility.splits(train,test_size=0.2)
trainx,trainy=features.feature_selection(train,multi=False)
valx,valy=features.feature_selection(val,multi=False)
testx,testy=features.feature_selection(test,multi=False)



param_space=optimization.randf_clf_params
model=models.pipeline_rfclf
bayes=optimization.bayes_estimation(param_space,trainx,trainy,valx,valy,model,False)
parameters=bayes.parameters(multi=False)
parameters=optimization.format_result(parameters)
model=model[-1].set_params(**parameters)
model.fit(trainx,trainy)
tr_preds=model.predict(trainx)
vl_preds=model.predict(valx)
ts_preds=model.predict(testx)

print(models.Train.single_out(tr_preds,trainy),
      models.Train.single_out(vl_preds,valy),
      models.Train.single_out(ts_preds,testy))
      

100%|██████████| 10/10 [00:09<00:00,  1.10trial/s, best loss: 0.5708898944193062]
0.8629852996607614 0.579185520361991 0.5781532890766445


In [231]:
# select only the best of features with mutual information a lot
# othe feature engineering techniques
import preprocessing
new_data2=preprocessing.season_datetime_fix(new_data.copy())

In [263]:
import datetime as dt
new_data2[['dayofseason','week','day']]=new_data2[['dayofseason','week','day']].astype('int64')